# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Initialize the WebDriver

In [2]:
# Initialize WebDriver (Adjust path if necessary)
driver = webdriver.Chrome()  # For Chrome

# Navigate to the Webpage

In [3]:
url = "https://community.sw.siemens.com/s/topic/0TO4O000000MiilWAC/nx-cam?tabset-d9298=2"
driver.get(url)

In [6]:
# Expand List
for i in range(100):
    try:
        # Wait for the "View More" button to be present
        time.sleep(1)  # Adjust sleep time if necessary
        view_more_button = driver.find_element(By.XPATH, "//button[text()='Load more']")
        view_more_button.click()
        print("Clicked 'Load More' button")
    except Exception as e:
        print("No more 'Load More' button found or an error occurred:", e)
        break

Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
No more 'Load More' button found or an error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[text()='Load more']"}
  (Session info: chrome=128.0.6613.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF683579412+29090]
	(No symbol) [0x00007FF6834EE239]
	(No symbol) [

# Collecting Links

In [8]:
def extract_links(driver, links):
    # Find the <ul> with class 'compactFeedList'
    ul_element = driver.find_element(By.XPATH, "//ul[@class=' article-list ']")
    li_elements = ul_element.find_elements(By.XPATH, "./li[contains(@class, 'article-item')]")

    for li in li_elements:
        # Find all <a> tags within the <li>
        a_tags = li.find_elements(By.TAG_NAME, "a")
        for a_tag in a_tags:
            link = a_tag.get_attribute("href")
            if link and link not in links:
                links.append(link)
    return links

links = []
extract_links(driver, links)

['https://community.sw.siemens.com/s/article/Introducing-the-NC-Optimizer-A-New-Approach-to-Postprocessing',
 'https://community.sw.siemens.com/s/article/Enhanced-Access-to-Information-with-NC-Optimizer',
 'https://community.sw.siemens.com/s/article/Tool-Path-Type-Changes-with-NC-Optimizer',
 'https://community.sw.siemens.com/s/article/What-s-new-in-Post-Hub-2024',
 'https://community.sw.siemens.com/s/article/Best-Practice-for-Creating-Multi-Tools-in-NX-CAM',
 'https://community.sw.siemens.com/s/article/Manufacturing-Resource-Library-for-Inspection',
 'https://community.sw.siemens.com/s/article/Commissioning-Manager-for-NX-CAM',
 'https://community.sw.siemens.com/s/article/Process-Patterning-for-Multi-Part-Programming-on-a-Tombstone-in-NX-CAM',
 'https://community.sw.siemens.com/s/article/Machine-Powered-Programming-for-the-regular-NX-CAM-user',
 'https://community.sw.siemens.com/s/article/Machine-Powered-Programming-for-advanced-Machine-Kit-Creators-in-NX-CAM',
 'https://community.sw.

In [9]:
def save_links_to_file(links, filename="article_links.txt"):
    with open(filename, "w") as file:
        for link in links:
            file.write(link + "\n")
    print(f"Saved {len(links)} links to {filename}")

save_links_to_file(links)

Saved 525 links to article_links.txt


# Going through all links

In [15]:
# Open the file in read mode
with open('article_links.txt', 'r') as file:
    # Read each line and strip any newline characters
    links = [line.strip() for line in file]

# Print the list of links to verify
print(links)

['https://community.sw.siemens.com/s/article/Introducing-the-NC-Optimizer-A-New-Approach-to-Postprocessing', 'https://community.sw.siemens.com/s/article/Enhanced-Access-to-Information-with-NC-Optimizer', 'https://community.sw.siemens.com/s/article/Tool-Path-Type-Changes-with-NC-Optimizer', 'https://community.sw.siemens.com/s/article/What-s-new-in-Post-Hub-2024', 'https://community.sw.siemens.com/s/article/Best-Practice-for-Creating-Multi-Tools-in-NX-CAM', 'https://community.sw.siemens.com/s/article/Manufacturing-Resource-Library-for-Inspection', 'https://community.sw.siemens.com/s/article/Commissioning-Manager-for-NX-CAM', 'https://community.sw.siemens.com/s/article/Process-Patterning-for-Multi-Part-Programming-on-a-Tombstone-in-NX-CAM', 'https://community.sw.siemens.com/s/article/Machine-Powered-Programming-for-the-regular-NX-CAM-user', 'https://community.sw.siemens.com/s/article/Machine-Powered-Programming-for-advanced-Machine-Kit-Creators-in-NX-CAM', 'https://community.sw.siemens.co

In [33]:
def extract_data_from_link(driver, link):
    driver.get(link)
    time.sleep(10)  # Wait for the page to load completely
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract question title
    title_div = soup.find("dd", class_="slds-form-element__control slds-grid itemBody")
    if title_div:
        article_title = title_div.get_text(strip=True)
    else:
        article_title = ""
    
    # Extract question body
    body_div = soup.find("div", class_="slds-rich-text-editor__output uiOutputRichText forceOutputRichText selfServiceOutputRichTextWithSmartLinks")
    if body_div:
        article_body = body_div.get_text(strip=True)
    else:
        article_body = ""
    
    # Return the data as a dictionary
    data = {
        'link': link,
        'article_title': article_title,
        'article_body': article_body,
    }
    return data

In [37]:
all_data = []

def collect_all_data(driver, links):
    for idx, link in enumerate(links):
        print(f"Processing link {idx+1}/{len(links)}: {link}")
        data = extract_data_from_link(driver, link)
        all_data.append(data)
        # if idx > 5:
        #     break
    return all_data

In [26]:
def save_data_to_csv(data_list, filename="article_data.csv"):
    # Prepare data for DataFrame
    rows = []
    for data in data_list:
        rows.append({
            'Link': data['link'],
            'Article Title': data['article_title'],
            'Article Body': data['article_body'],
        })
    # Create DataFrame
    df = pd.DataFrame(rows)
    # Save to CSV
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"Saved data to {filename}")

In [38]:
test_data = collect_all_data(driver, links)

Processing link 1/525: https://community.sw.siemens.com/s/article/Introducing-the-NC-Optimizer-A-New-Approach-to-Postprocessing
Processing link 2/525: https://community.sw.siemens.com/s/article/Enhanced-Access-to-Information-with-NC-Optimizer
Processing link 3/525: https://community.sw.siemens.com/s/article/Tool-Path-Type-Changes-with-NC-Optimizer
Processing link 4/525: https://community.sw.siemens.com/s/article/What-s-new-in-Post-Hub-2024
Processing link 5/525: https://community.sw.siemens.com/s/article/Best-Practice-for-Creating-Multi-Tools-in-NX-CAM
Processing link 6/525: https://community.sw.siemens.com/s/article/Manufacturing-Resource-Library-for-Inspection
Processing link 7/525: https://community.sw.siemens.com/s/article/Commissioning-Manager-for-NX-CAM
Processing link 8/525: https://community.sw.siemens.com/s/article/Process-Patterning-for-Multi-Part-Programming-on-a-Tombstone-in-NX-CAM
Processing link 9/525: https://community.sw.siemens.com/s/article/Machine-Powered-Programming

In [39]:
save_data_to_csv(test_data)

Saved data to article_data.csv
